In [37]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from pybaseball import statcast, cache
#from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import joblib
import math
from pybaseball import statcast
import scipy.stats as stats
#from catboost import Pool
#import optuna
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score, precision_score
import sqlite3
cache.enable()

In [38]:
data = pd.read_csv('savant_data.csv')

In [39]:
data['velocity'].mean()

np.float64(87.48987314085741)

In [40]:
# First, let's select the key features for the "Stuff+" model as discussed:
# - 'pitch_type'
# - 'pitch_percent'
# - 'swing_miss_percent'
# - 'arm_angle'
# - 'xbadiff'
# - 'xobp'
# - 'xslg'
# - 'pitcher_run_value_per_100'
# - 'batter_run_value_per_100'

# We will also select 'swing_miss_percent' as our initial target variable

# Selecting the relevant features
selected_columns = [
    'pitch_type', 'pitch_percent', 'swing_miss_percent', 'arm_angle', 
    'xbadiff', 'xobp', 'xslg', 'pitcher_run_value_per_100', 'batter_run_value_per_100'
]

data = data.dropna(subset=selected_columns)

# Extracting the features and target variable
features = data[selected_columns]
target = data['swing_miss_percent']

# One-hot encoding the categorical 'pitch_type' column to handle it in the model
features_encoded = pd.get_dummies(features, columns=['pitch_type'], drop_first=True)

# Splitting the data into training and testing sets for modeling
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_encoded, target, test_size=0.2, random_state=42)

# We will now train an initial regression model. I'll start with a Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialize the model
rf_model = RandomForestRegressor(random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Calculate the mean squared error for performance evaluation
mse = mean_squared_error(y_test, y_pred)
mse


np.float64(0.07160777923497333)

In [41]:
comparison = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})

print(comparison)

      Actual  Predicted
538     33.2     33.201
1822    41.0     41.007
534     51.0     50.888
2119    18.2     18.200
561     20.0     20.000
...      ...        ...
744     40.0     40.000
733     33.3     33.300
2647    28.6     28.600
4149    11.3     11.306
3819    27.3     27.300

[915 rows x 2 columns]


In [42]:
#mean_whiff = data['swing_miss_percent'].mean()
#data['Stuff+'] = (data['swing_miss_percent'] / mean_whiff) * 100

# Normalizing the columns
data['SwingMiss_norm'] = (data['swing_miss_percent'] / data['swing_miss_percent'].mean()) * 100
data['xwOBA_norm'] = (1 - (data['xwoba'] / data['xwoba'].max())) * 100 # Inverse
data['RunExp_norm'] = (1 - (data['run_exp'] / data['run_exp'].max())) * 100  # Inverse

# Assigning weights
w1, w2, w3 = .3, .2, .5

# Calculating Stuff+
data['Stuff+'] = (
    w1 * data['SwingMiss_norm'] +
    w2 * data['xwOBA_norm'] +
    w3 * data['RunExp_norm']
)

# Display the first few rows of the Stuff+ calculation
data[['swing_miss_percent', 'xobp', 'pitcher_run_value_per_100', 'Stuff+']].head()


,swing_miss_percent,xobp,pitcher_run_value_per_100,Stuff+
0,44.9,0.234,1.495762,189.707922
1,25.7,0.311,1.273859,172.284705
2,39.5,0.261,1.302039,171.132300
3,26.0,0.285,1.117130,156.173976
4,28.9,0.281,1.143414,155.772464


In [43]:
# Sort the DataFrame by the 'Stuff+' column in descending order (higher Stuff+ is better)
features_sorted = data.sort_values(by='Stuff+', ascending=False)
features_sorted.to_csv('savant_data.csv', index=False)

In [44]:
'''
data['SwingMiss_norm'] = (data['swing_miss_percent'] / data['swing_miss_percent'].mean()) * 100
data['xwOBA_norm'] = ((data['xwoba'] / data['xwoba'].mean())) * 100 # Inverse
data['RunExp_norm'] = ((data['run_exp'] / data['run_exp'].mean())) * 100  # Inverse

# Assigning weights
w1, w2, w3 = .3, .2, .5

# Calculating Stuff+
data['Stuff+'] = (
    w1 * data['SwingMiss_norm'] +
    w2 * data['xwOBA_norm'] +
    w3 * data['RunExp_norm']
)

# Display the first few rows of the Stuff+ calculation
data[['swing_miss_percent', 'xobp', 'pitcher_run_value_per_100', 'Stuff+']].head()
'''

"\ndata['SwingMiss_norm'] = (data['swing_miss_percent'] / data['swing_miss_percent'].mean()) * 100\ndata['xwOBA_norm'] = ((data['xwoba'] / data['xwoba'].mean())) * 100 # Inverse\ndata['RunExp_norm'] = ((data['run_exp'] / data['run_exp'].mean())) * 100  # Inverse\n\n# Assigning weights\nw1, w2, w3 = .3, .2, .5\n\n# Calculating Stuff+\ndata['Stuff+'] = (\n    w1 * data['SwingMiss_norm'] +\n    w2 * data['xwOBA_norm'] +\n    w3 * data['RunExp_norm']\n)\n\n# Display the first few rows of the Stuff+ calculation\ndata[['swing_miss_percent', 'xobp', 'pitcher_run_value_per_100', 'Stuff+']].head()\n"

In [45]:
print(data['xwoba'].min())
print(data['xwoba'].max())
print(data['xwoba'].mean())
print(data['pitcher_run_exp'].min())
print(data['pitcher_run_exp'].max())
print(data['pitcher_run_exp'].mean())


0.0
1.989
0.3247451881014873
-53.1
79.3
0.2558180227471567


In [46]:
# different normailzation so all are around 60-150, average at 100
data['pre_new_all_+'] = data['pitcher_run_exp'] - data['pitcher_run_exp'].min()
print(data['pre_new_all_+'].mean())

53.35581802274715


In [63]:
std_dev_pre = data['pitcher_run_exp'].std()
std_dev_xwoba = data['xwoba'].std()
std_dev_swm = data['swing_miss_percent'].std()

# try with standard dev
data['SwingMiss_norm'] = ((data['swing_miss_percent'] - data['swing_miss_percent'].mean()) / std_dev_swm) * 10
data['xwOBA_norm'] = (((data['xwoba'].mean() - data['xwoba'])) / std_dev_xwoba) * 10
data['RunExp_norm'] = ((data['pitcher_run_exp'] - data['pitcher_run_exp'].mean()) / std_dev_pre) * 5

# Assigning weights
w1, w2, w3 = .2, .3, .5

# Calculating Stuff+
data['Stuff+'] = (
    w1 * data['SwingMiss_norm'] +
    w2 * data['xwOBA_norm'] +
    w3 * data['RunExp_norm']
)

data['Stuff+'] = (data['Stuff+'] + 100)

# Display the first few rows of the Stuff+ calculation
data[['swing_miss_percent', 'xwoba', 'pitcher_run_exp', 'Stuff+']].head()

,swing_miss_percent,xwoba,pitcher_run_exp,Stuff+
0,44.9,0.225,72.7,127.638639
1,25.7,0.300,79.3,124.256708
2,39.5,0.243,60.0,122.426050
3,26.0,0.299,61.8,119.119853
4,28.9,0.280,57.5,118.837784


In [64]:
features_sorted = data.sort_values(by='Stuff+', ascending=False)
print(data['Stuff+'].mean())
features_sorted.to_csv('savant_data.csv', index=False)

100.0


In [65]:
filtered_data = data[data['pitches'] >= 200]
filtered_data = filtered_data.sort_values(by='Stuff+', ascending=False)
filtered_data.to_csv('STUFF+Initial.csv', index=False)

In [66]:
print(filtered_data[filtered_data['pitch_type'] == 'FC'].head())

    pitches  player_id         player_name  total_pitches pitch_type  \
1      6224     669203      Burnes, Corbin          11943         FC   
31     1278     621237      Alvarado, José           3720         FC   
12     2802     661403     Clase, Emmanuel           4080         FC   
87      318     596112  Stephenson, Robert           2444         FC   
88     1489     677651        Garcia, Luis           5435         FC   

    pitch_percent     ba    iso  babip    slg  ...  RunExp_norm  \
1            52.1  0.227  0.115  0.278  0.341  ...    47.125113   
31           34.4  0.136  0.063  0.327  0.199  ...    14.397552   
12           68.7  0.212  0.071  0.259  0.283  ...    27.870100   
87           13.0  0.101  0.152  0.143  0.253  ...     6.945921   
88           27.4  0.167  0.119  0.237  0.286  ...    12.132256   

    pre_new_all_+   ls_norm    ba_norm  Unhittable+  release_pos_x_norm  \
1           132.4  0.112477   2.438914   101.009848                0.77   
31           7

In [67]:
std_dev_ba = data['ba'].std()
std_dev_ls = data['launch_speed'].std()
std_dev_swm = data['swing_miss_percent'].std()

# try with standard dev
data['SwingMiss_norm'] = ((data['swing_miss_percent'] - data['swing_miss_percent'].mean()) / std_dev_swm) * 10
data['ls_norm'] = ((data['launch_speed'].mean() - data['launch_speed']) / std_dev_ls) * 10
data['ba_norm'] = ((data['ba'].mean() - data['ba']) / std_dev_ba) * 10

# Assigning weights
w1, w2, w3 = 1/3, 1/3, 1/3

# Calculating Stuff+
data['Unhittable+'] = (
    w1 * data['SwingMiss_norm'] +
    w2 * data['ls_norm'] +
    w3 * data['ba_norm']
)

data['Unhittable+'] = (data['Unhittable+'] + 100)

# Display the first few rows of the Stuff+ calculation
data[['swing_miss_percent', 'ba', 'launch_speed', 'Unhittable+']].head()

,swing_miss_percent,ba,launch_speed,Unhittable+
0,44.9,0.169,86.3,109.318822
1,25.7,0.227,87.8,100.988152
2,39.5,0.169,85.6,108.188003
3,26.0,0.211,91.3,98.992823
4,28.9,0.211,89.0,101.504980


In [68]:
filtered_data_unhit = data[data['pitches'] >= 200]
filtered_data_unhit = filtered_data_unhit.sort_values(by='Unhittable+', ascending=False)
filtered_data_unhit.to_csv('Unhittable+Initial.csv', index=False)
data['Unhittable+'].mean()

np.float64(100.00000000000001)

In [69]:
data['release_pos_x_norm'] = data['release_pos_x']
data['release_pos_x_norm'] = data['release_pos_x_norm'].apply(lambda x: -x if x < 0 else x)

# Filter for fastballs
fastball_df = data[data['pitch_type'] == 'FF']

# Group by pitcher and calculate average x_pos and z_pos for fastballs
fastball_positions = fastball_df.groupby('player_name')[['release_pos_x_norm', 'release_pos_z']].mean().reset_index()
fastball_positions.rename(columns={'release_pos_x_norm': 'fastball_x', 'release_pos_z': 'fastball_z'}, inplace=True)

# Step 2: Merge fastball positions into the original dataset
data = data.merge(fastball_positions, on='player_name', how='left')

# Step 3: Calculate differences for x_pos and z_pos
data['x_diff'] = data['release_pos_x_norm'] - data['fastball_x']
data['z_diff'] = data['release_pos_z'] - data['fastball_z']

# Step 4: Filter for non-fastball pitches
secondary_pitches = data[data['pitch_type'] != 'FF']

# Step 5: Keep only relevant columns for the result
result = secondary_pitches[['player_name', 'pitch_type', 'x_diff', 'z_diff']]

# Output the result
print(result.head())

KeyError: 'fastball_x'

In [71]:
print(data['release_pos_x_norm'].mean())
print(data[data['player_name'] == 'Valdez, Framber']['release_pos_x_norm'].mean())

1.905190735694823
0.8633333333333333


In [72]:
# Stuff Model!
# First, let's select the key features for the "Stuff+" model as discussed:
# - 'pitch_type'
# - 'velocity'
# - 'api_break_z_induced'
# - 'api_break_x_arm'
# - 'release_extension'
# - 'spin_rate'
# - 'release_pos_x'
# - 'release_pos_z'
# - 'x_diff'  
# - 'z_diff'
# - 'Stuff+'

# Adding New Variables, finding primary fastball
#if (

# Selecting the relevant features
selected_columns = [
    'pitch_type', 'velocity', 'api_break_z_induced', 'api_break_x_arm', 'release_extension', 'spin_rate', 'release_pos_x_norm', 'release_pos_z', 'x_diff', 'z_diff'  
]

data = data.dropna(subset=selected_columns)

# Extracting the features and target variable
features = data[selected_columns]
target = data['Stuff+']

# One-hot encoding the categorical 'pitch_type' column to handle it in the model
features_encoded = pd.get_dummies(features, columns=['pitch_type'], drop_first=True)

# Splitting the data into training and testing sets for modeling
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_encoded, target, test_size=0.2, random_state=42)

# We will now train an initial regression model. I'll start with a Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialize the model
rf_model = RandomForestRegressor(random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Calculate the mean squared error for performance evaluation
mse = mean_squared_error(y_test, y_pred)
mse


np.float64(25.80874490869251)

In [73]:
comparison = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})

print(comparison)
print(y_test.max())
print(y_pred.max())
print(y_test.mean())
print(y_pred.mean())
print(y_test.min())
print(y_pred.min())

          Actual   Predicted
3116   97.330498   98.977652
668   105.809101  108.648341
1213  103.803771   98.748846
478   106.686364  104.233509
530   104.211852   97.975168
...          ...         ...
4197   91.721684   99.675039
321   104.687517   98.832541
3043   96.829061   96.638490
888   104.220295  100.371009
655   105.714329  104.447658

[881 rows x 2 columns]
117.23563962779001
108.64834114796098
100.16132551149428
99.95212323514984
56.194876181621865
92.51639420106278


In [74]:
my_slider = [82, -.08333, -.91667, 6.5, 2800, 2.55, 6.02, 0, 0, False, False, False, False, False, False, False, False, False, False, True, False, False]
#a_slider = rf_model.predict([my_slider], columns=features_encoded)

import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names")
a_slider = rf_model.predict([my_slider])
print(a_slider)

[100.83916712]


In [75]:
my_fastball = [92, 1.4, 1.2, 6.5, 2300, 2.55, 6.02, 0, 0, False, False, False, True, False, False, False, False, False, False, False, False, False]
#a_slider = rf_model.predict([my_slider], columns=features_encoded)

import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names")
a_fastball = rf_model.predict([my_fastball])
print(a_fastball)

[98.80607619]


In [76]:
# Importing XGBoost
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Initialize the XGBoost model
xgb_model = XGBRegressor(
    random_state=42,  # Ensure reproducibility
    n_estimators=300,  # Number of trees (default: 100)
    learning_rate=0.01,  # Step size shrinkage (default: 0.3)
    max_depth=4,       # Maximum tree depth (default: 6)
    objective='reg:squarederror'  # Regression objective
)

# Train the XGBoost model
xgb_model.fit(X_train, y_train)

# Predict on the test set
y_pred = xgb_model.predict(X_test)

# Calculate the mean squared error for performance evaluation
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 25.246404380314882


In [77]:
comparison = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})

print(y_test.max())
print(y_pred.max())
print(y_test.mean())
print(y_pred.mean())
print(y_test.min())
print(y_pred.min())

117.23563962779001
108.18373
100.16132551149428
99.98617
56.194876181621865
93.369804


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [4, 6, 8]
}

# Initialize the model
xgb_model = XGBRegressor(random_state=42, objective='reg:squarederror')

# Perform Grid Search
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X_train, y_train)

# Get the best parameters and model
best_params = grid_search.best_params_
print(f"Best Parameters: {best_params}")

best_xgb_model = grid_search.best_estimator_


Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=200; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=200; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=200; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=300; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=300; total time=   0.3s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=300; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=100; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=100; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=6, n_es

In [ ]:
print(data.head())